In [358]:
# GENERAL USE LIBRARY IMPORTS

import datetime as dt
from datetime import timedelta
import random
import random as rand
import warnings
from datetime import datetime
import time

import matplotlib.pyplot as plt
import numpy as np
import numpy.random
import pandas as pd
import os
import requests
from zipfile import ZipFile

warnings.filterwarnings('ignore')

In [74]:
# Create dictionary of team abbvs and names
team_abbvs = ['ATL', 'BOS', 'BRK', 'CHO', 'CHI', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOP', 'NYK',  'OKC', 'ORL', 'PHI', 'PHO', 'POR', 'SAC','SAS', 'TOR', 'UTA','WAS']
team_names = ['Atlanta Hawks','Boston Celtics','Brooklyn Nets','Charlotte Hornets','Chicago Bulls','Cleveland Cavaliers','Dallas Mavericks','Denver Nuggets','Detroit Pistons','Golden State Warriors','Houston Rockets','Indiana Pacers','Los Angeles Clippers','Los Angeles Lakers','Memphis Grizzlies','Miami Heat','Milwaukee Bucks','Minnesota Timberwolves','New Orleans Pelicans','New York Knicks','Oklahoma City Thunder','Orlando Magic','Philadelphia 76ers','Phoenix Suns','Portland Trail Blazers','Sacramento Kings','San Antonio Spurs','Toronto Raptors','Utah Jazz','Washington Wizards']
name_to_abbv = dict(zip(team_names,team_abbvs))

In [3]:
import requests
# Read the Data
url = 'https://www.sportsbookreviewsonline.com/scoresoddsarchives/nba-odds-2012-13/'

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url,headers=header)

season_game_results = pd.read_html(r.text)[0]

# Make first row Column Names
season_game_results.columns = season_game_results.iloc[0]
# Delete first row
season_game_results = season_game_results[1:]
# Drop columns
season_game_results.drop(columns=['Date', 'Rot','1st', '2nd', '3rd', '4th', 'Open', 'Close', '2H'], inplace=True)
# Rename Columns
column_name_changes = {
    'Team': 'Away_Team',
    'Final': 'A_Final_Score'}
season_game_results.rename(columns=column_name_changes, inplace=True)

season_game_results['Home_Team'] = season_game_results.apply(lambda row: row['Away_Team'] if row['VH'] == 'H' else '', axis=1).shift(-1)
season_game_results['H_Final_Score'] = season_game_results.apply(lambda row: row['A_Final_Score'] if row['VH'] == 'H' else '', axis=1).shift(-1)
season_game_results['ML'] = season_game_results['ML'].shift(-1)
season_game_results = season_game_results[~(season_game_results.index % 2 == 0)]
season_game_results['W/L'] = season_game_results.apply(lambda row: 'W' if row['H_Final_Score'] >= row['A_Final_Score'] else 'L', axis=1)

season_game_results = season_game_results.reset_index(drop=True)


# Display df
season_game_results

,VH,Away_Team,A_Final_Score,ML,Home_Team,H_Final_Score,W/L
0,V,Washington,84,-250,Cleveland,94,W
1,V,Boston,107,-280,Miami,120,W
2,V,Dallas,99,-500,LALakers,91,L
3,V,Indiana,90,100,Toronto,88,L
4,V,Denver,75,110,Philadelphia,84,W
...,...,...,...,...,...,...,...
1309,V,Miami,77,-125,SanAntonio,113,L
1310,V,Miami,109,110,SanAntonio,93,W
1311,V,Miami,104,-110,SanAntonio,114,W
1312,V,SanAntonio,100,-280,Miami,103,W


In [192]:
df_target = pd.DataFrame(columns=hometeam.columns)

# extract last row (team data)
new_row = hometeam.iloc[-1]
new_row_df = new_row.to_frame().T

# Optionally, reset the index if you don't want the original index
new_row_df.reset_index(drop=True, inplace=True)

df_target = pd.concat([df_target, new_row_df ], ignore_index=True)
print(df_target)

       Players   MP  FG FGA   FG%  3P 3PA   3P%  FT FTA  ...  DRB%  TRB%  \
0  Team Totals  240  39  82  .476  22  49  .449  14  16  ...  87.2  56.4   

   AST% STL%  BLK%  TOV%   USG%   ORtg   DRtg  BPM  
0  69.2  6.9  13.6  10.1  100.0  131.7  108.6  NaN  

[1 rows x 37 columns]


In [132]:
# team stats
team_stats = pd.DataFrame(columns = ['Team', 'FG%', '3P%', 'FT%', 'TOVs forced',
                                     'TOV', 'PPG', 'TS%', 'eFG%', 'TRB%', 'TOV%',
                                     'ORtg', 'DRtg'])



In [213]:
def get_boxscore(response):
    
    boxscore = pd.read_html(response)
    # differing layouts if ot
    num = len(boxscore)
    num_div_two = int(num/2)
    awayteam_basic = boxscore[0]
    awayteam_advanced = boxscore[num_div_two -1]
    hometeam_basic = boxscore[num_div_two]
    hometeam_advanced = boxscore[num-1]
    
    awayteam = pd.merge(awayteam_basic,awayteam_advanced,left_index= True,right_index =True)
    awayteam = awayteam.drop(5,axis = 0).drop(columns = ('Unnamed: 0_level_0_y','Starters'),axis = 1)
    awayteam = awayteam.rename(columns={'Starters': 'Players'})
    awayteam.columns = awayteam.columns.droplevel(0)
    
    hometeam = pd.merge(hometeam_basic,hometeam_advanced,left_index= True,right_index =True)
    hometeam = hometeam.drop(5,axis = 0).drop(columns = ('Unnamed: 0_level_0_y','Starters'),axis = 1)
    hometeam = hometeam.rename(columns={'Starters': 'Players'})
    hometeam.columns = hometeam.columns.droplevel(0)

    return awayteam, hometeam

In [356]:
# List of all months for NBA games in the season
months = ['october', 'november', 'december', 'january', 'february', 'march']

# Base URL for fetching data
base_url = "https://www.basketball-reference.com/leagues/NBA_2024_games-{}.html"

# Initialize an empty list to store DataFrames
full_schedule = []

# Loop through each month and fetch game data
for month in months:
    try:
        # Format URL for the current month
        url = base_url.format(month)
        
        # Read HTML table data
        month_schedule = pd.read_html(url)[0]  # Assumes the first table is the relevant one
        
        # Append the DataFrame to the full_schedule list
        full_schedule.append(month_schedule)
    except Exception as e:
        print(f"Failed to fetch data for {month}: {e}")

# Concatenate all DataFrames into one
full_schedule_df = pd.concat(full_schedule, ignore_index=True)

# Print or inspect the full schedule DataFrame
#print(full_schedule_df)

Failed to fetch data for october: HTTP Error 429: Too Many Requests
Failed to fetch data for november: HTTP Error 429: Too Many Requests
Failed to fetch data for december: HTTP Error 429: Too Many Requests
Failed to fetch data for january: HTTP Error 429: Too Many Requests
Failed to fetch data for february: HTTP Error 429: Too Many Requests
Failed to fetch data for march: HTTP Error 429: Too Many Requests


ValueError: No objects to concatenate

In [350]:
full_schedule_df.to_csv("full_scedule.csv", index = False)

In [69]:
full_schedule_df.dtypes
full_schedule_df['f_Date'] = pd.to_datetime(full_schedule_df['Date']).dt.strftime('%Y%m%d')

In [78]:
full_schedule_df['Home_abbr'] = full_schedule_df['Home/Neutral'].map(name_to_abbv)
full_schedule_df['Away_abbr'] = full_schedule_df['Visitor/Neutral'].map(name_to_abbv)


In [355]:
#print(full_schedule_df)

In [80]:
simple_games_df = full_schedule_df[['f_Date', 'Home_abbr', 'Away_abbr']].rename(columns={
    'f_Date': 'Date',
    'Home_abbr': 'Home',
    'Away_abbr': 'Away'
})

In [81]:
print(simple_games_df)

          Date Home Away
0     20231024  DEN  LAL
1     20231024  GSW  PHO
2     20231025  ORL  HOU
3     20231025  NYK  BOS
4     20231025  IND  WAS
...        ...  ...  ...
1111  20240331  HOU  DAL
1112  20240331  MIN  CHI
1113  20240331  NYK  OKC
1114  20240331  SAS  GSW
1115  20240331  SAC  UTA

[1116 rows x 3 columns]


In [397]:

# create list of dates:
# Define the start and end dates
start_date = '2023-10-24'
#testing with smaller set
#end_date = '2023-10-31'
end_date = '2024-01-30'

# Generate the range of dates
date_range = pd.date_range(start=start_date, end=end_date)

# Convert to list
date_list = list(date_range)

# format dates:
formatted_date_list = [date.strftime('%Y%m%d') for date in date_range]

# Print the formatted list of dates
print(formatted_date_list)
# Print the list of dates


['20231024', '20231025', '20231026', '20231027', '20231028', '20231029', '20231030', '20231031', '20231101', '20231102', '20231103', '20231104', '20231105', '20231106', '20231107', '20231108', '20231109', '20231110', '20231111', '20231112', '20231113', '20231114', '20231115', '20231116', '20231117', '20231118', '20231119', '20231120', '20231121', '20231122', '20231123', '20231124', '20231125', '20231126', '20231127', '20231128', '20231129', '20231130', '20231201', '20231202', '20231203', '20231204', '20231205', '20231206', '20231207', '20231208', '20231209', '20231210', '20231211', '20231212', '20231213', '20231214', '20231215', '20231216', '20231217', '20231218', '20231219', '20231220', '20231221', '20231222', '20231223', '20231224', '20231225', '20231226', '20231227', '20231228', '20231229', '20231230', '20231231', '20240101', '20240102', '20240103', '20240104', '20240105', '20240106', '20240107', '20240108', '20240109', '20240110', '20240111', '20240112', '20240113', '20240114', '20

In [218]:
def fetch_with_retry_after(url):
    response = requests.get(url)
    if response.status_code == 429:
        retry_after = int(response.headers.get('Retry-After', 60))  # Default to 60 seconds if header is missing
        print(f"Rate limit exceeded. Retrying after {retry_after} seconds...")
        time.sleep(retry_after)
        return fetch_with_retry_after(url)  # Recursively retry fetching
    response.raise_for_status()  # Raise an error for other status codes
    return response

In [224]:
def save_box_scores(formatted_date_list, simple_games_df, url_base):
    # Create a main directory to hold all data before zipping
    main_folder = 'NBA_Box_Scores'
    os.makedirs(main_folder, exist_ok=True)
    
    for date in formatted_date_list:
        games_df = simple_games_df[simple_games_df['Date'] == date]
        for index, row in games_df.iterrows():
            home_abbr = row['Home']
            away_abbr = row['Away']
            game_folder = f"{date}/{away_abbr}@{home_abbr}"  # Folder name format: YYYYMMDD/Away@Home
            full_folder_path = os.path.join(main_folder, game_folder)
            os.makedirs(full_folder_path, exist_ok=True)

            # Format the URL
            formatted_url = f"{url_base}{date}0{home_abbr}.html"
            # Fetch and save box scores
            try: 
                response = fetch_with_retry_after(formatted_url)
                
            # Save each team's box score in the specific game folder
                away_df,home_df = get_boxscore(response.text)
                away_df.to_csv(f"{full_folder_path}/away_team.csv", index=False)
                home_df.to_csv(f"{full_folder_path}/home_team.csv", index=False)
            except Exception as e:
                print(f"Error fetching data for URL {formatted_url}: {str(e)}")
            
    # Zip the entire directory
    with ZipFile(f"{main_folder}.zip", 'w') as zipf:
        for root, dirs, files in os.walk(main_folder):
            for file in files:
                zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), os.path.join(main_folder, '..')))

# Example usage
url_base = "https://www.basketball-reference.com/boxscores/"

save_box_scores(formatted_date_list, simple_games_df, url_base)

Rate limit exceeded. Retrying after 3600 seconds...


KeyboardInterrupt: 

In [201]:
all_players_stats = pd.DataFrame()

for date in formatted_date_list:
    games_df = simple_games_df[simple_games_df['Date'] == date]
    for index, row in games_df.iterrows():
        # Format the URL with the current game's home team abbreviation
        home_abbr = row['Home']
        formatted_url = "{}{}0{}.html".format(url_base, date, home_abbr)

        # Fetch the box score data
        awayteam, hometeam = get_boxscore(formatted_url)  # Assuming this function returns two DataFrames

        # Process both teams
        for team_df in [awayteam, hometeam]:
            team_df['Date'] = date  # Add date to track games
            team_df['Games Played'] = 1  # Initialize games played

            # Update player stats in the master DataFrame
            for i, player_row in team_df.iterrows():
                player_name = player_row['Players']
                
                if player_name in all_players_stats.index:
                    # Player exists, update their stats
                    player_stats = all_players_stats.loc[player_name]
                    new_games_played = player_stats['Games Played'] + 1
                    
                    # Calculate new averages for each stat
                    for col in player_stats.index:
                        if col not in ['Players', 'Date', 'Games Played']:
                            all_players_stats.at[player_name, col] = (player_stats[col] * player_stats['Games Played'] + player_row[col]) / new_games_played
                    
                    # Update games played
                    all_players_stats.at[player_name, 'Games Played'] = new_games_played
                else:
                    # New player, add to DataFrame
                    player_row = player_row.to_frame().T  # Transpose to convert Series to DataFrame
                    all_players_stats = pd.concat([all_players_stats, player_row], axis=0)

# Output the updated stats DataFrame
print(all_players_stats)

HTTPError: HTTP Error 429: Too Many Requests

## Team stats

In [ ]:
# for zips
def update_team_stats_from_zip(zip_path):
    # DataFrame to hold all team stats
    all_teams_stats = pd.DataFrame()

    # Open the zip file
    with ZipFile(zip_path, 'r') as zipf:
        # List all CSV files (assuming all box scores are stored as CSVs)
        boxscore_files = [file for file in zipf.namelist() if file.endswith('.csv')]

        # Process each CSV file
        for file_path in boxscore_files:
            # Extract data from the CSV file
            with zipf.open(file_path) as file:
                df = pd.read_csv(file)
                
                # Get the last row which represents team stats
                team_stats_row = df.iloc[-1]  # Access the last row directly
                
                team_name = team_stats_row['Team']
                if team_name in all_teams_stats.index:
                    # Update existing team stats and calculate new averages
                    update_running_averages(all_teams_stats, team_name, team_stats_row)
                else:
                    # Add new team
                    all_teams_stats = add_new_team(all_teams_stats, team_stats_row)

    return all_teams_stats


In [393]:
## update to change from zips
## for file names
def update_team_from_boxscore(data,date, away, home ): # path is  NBA_Box_Scores/date/away@home
    #data = pd.DataFrame()
    base_directory = "NBA_Box_Scores"
    directory_path = f"{base_directory}/{date}/{away}@{home}"
    boxscore_files = [f for f in os.listdir(directory_path) if f.endswith('.csv')]
    for file_name in boxscore_files:
        if file_name == "away_team.csv":
            team_name = away
        elif file_name == "home_team.csv":
            team_name = home
        else:
            print("error no csv found")
            continue
        file_path = os.path.join(directory_path, file_name)
        df = pd.read_csv(file_path)
        team_stats_row = df.iloc[-1]
        # convert objects into floats
        #numeric_columns = team_stats_row.drop(['Players'])  # Assuming 'Team' and 'Games Played' are not to be converted
        #team_stats_row[numeric_columns.index] = numeric_columns.astype(float)
        
        if team_name in data.index:
            data = update_running_team_averages(data, team_name, team_stats_row)
        else:
            # Add new team
            data = add_new_team(data,team_name, team_stats_row)

    return data
    
        

def update_running_team_averages(data, team_name, team_stats_row):
    """Update team stats and calculate running averages."""
    existing_stats = data.loc[team_name]
    games_played = existing_stats['Games Played'] + 1
    for col in team_stats_row.index:
        new_value = pd.to_numeric(team_stats_row[col], errors='coerce')
        old_avg = pd.to_numeric(existing_stats[col], errors='coerce')
        if pd.notna(new_value) and pd.notna(old_avg):
            new_avg = ((old_avg * (games_played - 1)) + new_value) / games_played
            data.at[team_name, col] = new_avg
    data.at[team_name, 'Games Played'] = games_played
    return data

def add_new_team(data, team_name, team_stats_row):
    """Add a new team to the DataFrame."""
    team_stats_row['Games Played'] = 1  # Initialize games played
    team_name = [team_name]
    team_stats_df = pd.DataFrame([team_stats_row], index = team_name)
    data = pd.concat([data, team_stats_df])
    return data

In [398]:
## creates cumul team stats for each date
all_team_stats = pd.DataFrame()
main_folder = "NBA_Team_Statsheet"
os.makedirs(main_folder, exist_ok = True)
for date in formatted_date_list:
    date_folder = f"{date}"
    full_folder_path = os.path.join(main_folder, date_folder)
    os.makedirs(full_folder_path, exist_ok = True)
    games_df = simple_games_df[simple_games_df['Date'] == date]
    for index, row in games_df.iterrows():
        # Format the URL with the current game's home team abbreviation
        home_abbr = row['Home']
        away_abbr = row['Away']
        

        all_team_stats = update_team_from_boxscore(all_team_stats,date,away_abbr,home_abbr)
    all_team_stats.to_csv(f"{full_folder_path}/statsheet.csv")
        
#print(all_team_stats)       

In [390]:
# get rid of null and unnessary values
def clean_team_stats(data):
    new_df = data.drop(['Players','MP','AST','+/-','MP.1','AST%','BPM'],axis = 1, inplace = True)
    return new
new_df = clean_team_stats(all_team_stats)
all_team_stats

,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,...,ORB%,DRB%,TRB%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,Games Played
DEN,47.25,87.75,0.53875,12.0,31.5,0.3795,9.75,15.0,0.66875,8.0,...,21.375,74.075,50.575,8.125,12.675,12.3,100.0,119.375,104.6,4
LAL,40.5,87.75,0.464,9.5,32.5,0.2865,19.5,25.25,0.7715,8.0,...,17.55,79.625,49.5,7.425,11.2,12.325,100.0,107.3,109.5,4
GSW,42.5,91.75,0.465,14.25,40.25,0.35925,16.25,21.25,0.769,13.5,...,28.4,74.15,52.425,9.15,9.7,12.65,100.0,115.925,104.975,4
PHO,40.333333,87.0,0.463667,10.0,33.666667,0.297667,19.0,22.666667,0.825667,11.333333,...,25.666667,73.866667,51.733333,7.333333,10.2,14.6,100.0,109.6,102.5,3
ORL,38.666667,87.0,0.446333,10.666667,32.333333,0.328667,19.0,25.333333,0.754333,13.0,...,29.933333,82.166667,55.366667,9.7,9.333333,11.766667,100.0,111.833333,100.466667,3
HOU,39.333333,89.666667,0.434333,9.666667,33.0,0.292,12.666667,18.333333,0.690667,9.333333,...,20.233333,70.4,43.8,6.733333,6.866667,12.7,100.0,101.1,116.866667,3
NYK,37.333333,92.333333,0.405333,15.0,40.666667,0.361,16.0,24.0,0.676667,15.0,...,28.333333,83.733333,52.966667,8.4,5.5,12.233333,100.0,106.333333,108.833333,3
BOS,44.333333,91.333333,0.485,15.666667,43.666667,0.358667,13.333333,17.333333,0.748,12.666667,...,28.333333,74.733333,53.5,8.066667,14.033333,13.133333,100.0,119.333333,108.966667,3
IND,46.666667,97.666667,0.474,15.666667,42.333333,0.373667,15.333333,18.0,0.860333,10.666667,...,24.033333,74.8,51.066667,6.333333,8.033333,10.2,100.0,120.7,111.9,3
WAS,41.0,92.0,0.445333,12.0,35.333333,0.343,19.333333,25.666667,0.752667,11.0,...,22.2,76.066667,46.966667,8.7,11.233333,11.433333,100.0,108.4,119.266667,3


## player stats

In [366]:
df = pd.read_csv("NBA_Box_Scores/20231024/PHO@GSW/away_team.csv")
player_rows = df.iloc[:-1]
        # convert objects into floats
#print(player_rows)

player_names = player_rows['Players'].tolist()
for i, player_row in player_rows.iterrows():
    player = player_row['Players']
    player_stats_row = player_rows.iloc[i]
    print(player)
    #print(player_stats_row)
    new_value = pd.to_numeric(player_stats_row['MP'], errors='coerce')
    ifplayed(player_stats_row)

Devin Booker
t
Kevin Durant
t
Josh Okogie
t
Jusuf Nurkić
t
Grayson Allen
t
Eric Gordon
t
Drew Eubanks
t
Yuta Watanabe
t
Jordan Goodwin
t
Nassir Little
t
Udoka Azubuike
f
Keita Bates-Diop
f
Bol Bol
f
Saben Lee
f
Chimezie Metu
f


In [372]:
## update to change from zips
## for file names
def update_player_from_boxscore(data, date, away, home ): # path is  NBA_Box_Scores/date/away@home
    #data = pd.DataFrame()
    base_directory = "NBA_Box_Scores"
    directory_path = f"{base_directory}/{date}/{away}@{home}"
    boxscore_files = [f for f in os.listdir(directory_path) if f.endswith('.csv')]
    for file_name in boxscore_files:
    
        file_path = os.path.join(directory_path, file_name)
        df = pd.read_csv(file_path)
        player_rows = df.iloc[:-1]

        player_names = player_rows['Players'].tolist()
        for i, player_row in player_rows.iterrows():
            player = player_row['Players']
            player_stats_row = player_rows.iloc[i]
            if player in data.index:
                data = update_running_player_averages(data, player_name, player_stats_row)
        else:
            # Add new team
            data = add_new_player(data,player_name, player_stats_row)

    return data
    
        

def update_running_player_averages(data, player_name, player_stats_row):
    """Update team stats and calculate running averages."""
    existing_stats = data.loc[player_name]
    if ifplayed(player_stats_row) == True:
        games_played = existing_stats['Games Played'] + 1
    else:
        return
    for col in player_stats_row.index:
        new_value = pd.to_numeric(player_stats_row[col], errors='coerce')
        old_avg = pd.to_numeric(player_stats[col], errors='coerce')
        if pd.notna(new_value) and pd.notna(old_avg):
            new_avg = ((old_avg * (games_played - 1)) + new_value) / games_played
            data.at[player_name, col] = new_avg
            
    data.at[player_name, 'Games Played'] = games_played
    return data

def add_new_player(data, player_name, player_stats_row):
    """Add a new team to the DataFrame."""
    if ifplayed(player_stats_row) == True:
        player_stats_row['Games Played'] = 1 # Initialize games played
    else: 
        return
    player_name = [player_name]
    player_stats_df = pd.DataFrame([player_stats_row], index = player_name)
    data = pd.concat([data, player_stats_df])
    return data



In [367]:
def ifplayed(player_stats_row):
    try:
        time_str = player_stats_row['MP']
        time_delta = datetime.strptime(time_str, "%M:%S") - datetime.strptime("00:00", "%M:%S")

        # Convert to total minutes as a float
        total_minutes = time_delta.total_seconds() / 60  # Divide by 60 to convert seconds to minutes

        player_stats_row['MP'] = total_minutes
        return player_stats_row
        return True
    except:

        return False
    

In [373]:
## creates cumul team stats for a date
all_player_stats = pd.DataFrame()
for date in formatted_date_list:
    games_df = simple_games_df[simple_games_df['Date'] == date]
    for index, row in games_df.iterrows():
        # Format the URL with the current game's home team abbreviation
        home_abbr = row['Home']
        away_abbr = row['Away']

        all_player_stats = update_player_from_boxscore(all_player_stats,date,away_abbr,home_abbr)
#print(all_team_stats)       

AttributeError: 'NoneType' object has no attribute 'index'